In [ ]:
!pip install aicrowd-cli


     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 174kB 25.5MB/s 
     |████████████████████████████████| 215kB 50.2MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


API Key valid
Saved API Key successfully!


In [ ]:
# Downloading the Dataset
!mkdir data


val.csv:   0% 0.00/262k [00:00<?, ?B/s]
train.csv:   0% 0.00/2.30M [00:00<?, ?B/s]

val.csv: 100% 262k/262k [00:00<00:00, 544kB/s]


test.csv: 100% 642k/642k [00:00<00:00, 970kB/s]

train.csv: 100% 2.30M/2.30M [00:00<00:00, 2.52MB/s]


In [ ]:
!pip install --upgrade transformers
!pip install simpletransformers
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize


     |████████████████████████████████| 2.5MB 4.3MB/s 
     |████████████████████████████████| 3.3MB 56.4MB/s 
     |████████████████████████████████| 901kB 65.5MB/s 
     |████████████████████████████████| 225kB 4.3MB/s 
     |████████████████████████████████| 122kB 50.0MB/s 
     |████████████████████████████████| 1.8MB 43.6MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 245kB 58.9MB/s 
     |████████████████████████████████| 7.8MB 57.4MB/s 
     |████████████████████████████████| 1.2MB 43.7MB/s 
     |████████████████████████████████| 102kB 11.9MB/s 
     |████████████████████████████████| 133kB 41.2MB/s 
     |████████████████████████████████| 122kB 50.2MB/s 
     |████████████████████████████████| 245kB 58.0MB/s 
     |████████████████████████████████| 81kB 14.3MB/s 
     |████████████████████████████████| 4.2MB 52.4MB/s 
     |████████████████████████████████| 112kB 55.5MB/s 
     |████████████████████████████████| 122kB 61.8MB/

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=61d9149b284623e389a1ee76b406e276131b805bb44e858e96322a000ac5befc
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


Gen RAM Free: 26.3 GB  |     Proc size: 116.6 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total     16280MB


In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
import re
import random
import torch
pd.options.display.max_colwidth = 200

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False


In [ ]:
def preprocess(text):
  text  =  re.sub("@[A-Za-z0-9]+","", text)
  text  =  re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text  =  text.strip()
  return text



In [ ]:
import pandas as pd
train_df  = pd.read_csv("/content/data/train.csv" )
print(train_df.shape)
train_df  = train_df.dropna()
print(train_df.shape)
val_df  = pd.read_csv("/content/data/val.csv" )
print(val_df.shape)
val_df  = val_df.dropna()
print(val_df.shape)

test_df = pd.read_csv('/content/data/test.csv')
print(test_df.shape)
train_df.head()

(31255, 2)
(31255, 2)
(3473, 2)
(3473, 2)
(8682, 2)


,text,label
0,takes no time to copy/paste a press release,0
1,You're delusional,1
2,"Jazz fan here. I completely feel. Lindsay Mann cousins has more votes than Lindsay Mann, and Lindsay Mann hasn't even stepped on the court this year",0
3,ah i was also confused but i think they mean friends around the same age,0
4,Thank you so much. ♥️ that means a lot.,0


In [ ]:
val_df.head()

,text,label
0,While I agree with my political views could be of the top of my least favorites and Anthony Russell weren't ruined by the pic.,0
1,im still starving,1
2,*Hey just noticed..* it's your **2nd Cakeday** slumbishop! ^(hug),0
3,They just did. Check out the sticky post.,0
4,"I hope so too, she deserves it.",0


In [ ]:
train_df=pd.concat([train_df,val_df],axis=0)
train_df.head()

,text,label
0,takes no time to copy/paste a press release,0
1,You're delusional,1
2,"Jazz fan here. I completely feel. Lindsay Mann cousins has more votes than Lindsay Mann, and Lindsay Mann hasn't even stepped on the court this year",0
3,ah i was also confused but i think they mean friends around the same age,0
4,Thank you so much. ♥️ that means a lot.,0


In [ ]:
train_df.label.value_counts(normalize=True)

0    0.79086
1    0.20914
Name: label, dtype: float64

In [ ]:
val_df.label.value_counts(normalize=True)

0    0.790959
1    0.209041
Name: label, dtype: float64

In [ ]:
train_df.text.apply(lambda x:len(x.split())).describe()

count    34728.000000
mean        13.015521
std          6.780115
min          1.000000
25%          7.000000
50%         12.000000
75%         18.000000
max         36.000000
Name: text, dtype: float64

In [ ]:
test_df.text.apply(lambda x:len(x.split())).describe()

count    8682.000000
mean       13.075098
std         6.757223
min         1.000000
25%         7.000000
50%        13.000000
75%        18.000000
max        32.000000
Name: text, dtype: float64

In [ ]:
train_df['target'] = train_df['label'].values
test_df['target'] = 0

train1=train_df[['text','target']]
test1=test_df[['text','target']]
test1['target']=0

In [12]:
err=[]
y_pred_tot=[]

fold=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(train1,train1['target']):
    train1_trn, train1_val = train1.iloc[train_index], train1.iloc[test_index]
    model = ClassificationModel('roberta', 'roberta-base', use_cuda=True,num_labels=2, args={'train_batch_size':16,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': True,
                                                                         'do_lower_case': True,
                                                                         'num_train_epochs': 5,
                                                                         'max_seq_length': 36,
                                                                         'regression': False,
                                                                         'manual_seed': 42,
                                                                         "learning_rate":3e-5,
                                                                         'weight_decay':0,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,
                                                                         "silent": True})
    model.train_model(train1_trn)
    raw_outputs_val = model.eval_model(train1_val)[1]
    raw_outputs_val = softmax(raw_outputs_val,axis=1)[:,1]>0.5
    print(f"Accuracy: {accuracy_score(train1_val['target'], raw_outputs_val)}")
    print(f"F Score: {f1_score(train1_val['target'], raw_outputs_val)}")
    err.append(log_loss(train1_val['target'], raw_outputs_val))
    raw_outputs_test = model.eval_model(test1)[1]
    raw_outputs_test = softmax(raw_outputs_test,axis=1)[:,1]
    y_pred_tot.append(raw_outputs_test)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out

Accuracy: 0.8626547653325655
F Score: 0.6774847870182557


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out

Accuracy: 0.8491217967175353
F Score: 0.645945945945946


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out

...

Accuracy: 0.8609271523178808
F Score: 0.6743088334457182


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out

Accuracy: 0.8571428571428571
F Score: 0.6507042253521127


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out

Accuracy: 0.8533986175115207
F Score: 0.6581598388179987


In [13]:
del test_df['target']
test_df.head(2)

,text,label
0,I was already over the edge with Cassie Zamora. Just showing my disdain for two terrible individuals.,0
1,I think you're right. She has oodles of cash and young grandchildren to enjoy. Going through that hideous gauntlet again probably isn't that appealing.,0


In [14]:
test_df['label']=np.mean(y_pred_tot, 0)
test_df['label']= np.where(test_df['label']>0.5,1,0)
test_df.head()

,text,label
0,I was already over the edge with Cassie Zamora. Just showing my disdain for two terrible individuals.,1
1,I think you're right. She has oodles of cash and young grandchildren to enjoy. Going through that hideous gauntlet again probably isn't that appealing.,0
2,Haha I love this. I used to give mine phone books and my room would look just like this in a matter of hours. Crazy.,0
3,Probably out of desperation as they going no answers with the other made up god.,0
4,Sorry !! You’re real good at that!!,0


In [15]:
test_df.label.value_counts(normalize=True)

0    0.788874
1    0.211126
Name: label, dtype: float64

In [16]:
!mkdir assets

# Saving the sample submission in assets directory
test_df.to_csv(os.path.join("assets", "submission.csv"), index=False)

mkdir: cannot create directory ‘assets’: File exists


Using notebook: /content/drive/MyDrive/Colab Notebooks/Emotion_Detection.ipynb for submission...
Removing existing files from submission directory...
Scrubbing API keys from the notebook...
submission.zip ━━━━━━━━━━━━━━━━━━ 100.0% • 312.3/310.7 KB • 380.2 kB/s • 0:00:00
                                                  ╭─────────────────────────╮                                                  
                                                  │ Successfully submitted! │                                                  
                                                  ╰─────────────────────────╯                                                  
                                                        Important links                                                        
┌──────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/ai-blitz-9/problems/emotion-detec